In [1]:
!pip install elasticsearch langchain google-cloud-aiplatform

  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 9.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
   ----------------- ---------------------- 3.4/7.6 MB 16.7 MB/s eta 0:00:01
   ---------------------------------------- 7.6/7.6 MB 27.6 MB/s eta 0:00:00
Using cached packaging-24.2-py3-none-any.whl (65 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 30.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 29.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 4.3/4.3 MB 36.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install -U langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------- ----------------------- 1.0/2.5 MB 8.4 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 6.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install --upgrade langchain langchain-community


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
!pip install -qU langchain-google-vertexai


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import os

# remove it if present
os.environ.pop("GOOGLE_APPLICATION_CREDENTIALS", None)


In [ ]:
import os

# 2) Point to your service-account JSON
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "E:/HUST/20242/IT4930-DS/Project/TrendFinGPT/chatbot/ds-project-459819-000ad11e9d0c.json"
)

In [3]:
from elasticsearch import Elasticsearch, helpers
from langchain_google_vertexai import VertexAIEmbeddings
from google.oauth2 import service_account
import vertexai
import os

# 2) Point to your service-account JSON
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "E:/HUST/20242/IT4930-DS/Project/TrendFinGPT/chatbot/ds-project-459819-1e8f421d6a80.json"
)

# 2) Globally initialize Vertex AI with those creds
vertexai.init(
    project="ds-project-459819",
    location="us-central1",
)

# 3) Now build the LangChain embedder (it will inherit the above init)
embedder = VertexAIEmbeddings(
    model_name="text-multilingual-embedding-002",   # or "text-embedding-004"
)

# 4) Connect to Elasticsearch
es = Elasticsearch("http://localhost:9200")

# 5) Scroll & embed loop
scroll = es.search(
    index="vietnamnet",
    body={"query": {"match_all": {}}, "_source": ["summary","content"]},
    scroll="2m",
    size=500
)
sid = scroll["_scroll_id"]
docs = scroll["hits"]["hits"]

while docs:
    texts = [
        d["_source"].get("summary","") or d["_source"].get("content","")
        for d in docs
    ]
    vectors = embedder.embed_documents(texts)

    actions = [
      {
        "_op_type": "update",
        "_index": doc["_index"],
        "_id":    doc["_id"],
        "doc":    {"vector": vec}
      }
      for doc, vec in zip(docs, vectors)
    ]
    helpers.bulk(es, actions)

    scroll = es.scroll(scroll_id=sid, scroll="2m")
    sid = scroll["_scroll_id"]
    docs = scroll["hits"]["hits"]

print("All documents updated.")


Unauthenticated: 401 Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project. [reason: "ACCESS_TOKEN_EXPIRED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "method"
  value: "google.cloud.aiplatform.v1.ModelGardenService.GetPublisherModel"
}
]

In [5]:
!pip install sentence-transformers

  Using cached pillow-11.2.1-cp312-cp312-win_amd64.whl.metadata (9.1 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   - -------------------------------------- 6.8/212.5 MB 35.0 MB/s eta 0:00:06
   -- ------------------------------------- 15.5/212.5 MB 46.4 MB/s eta 0:00:05
   ---- ----------------------------------- 22.8/212.5 MB 38.0 MB/s eta 0:00:05
   ------ --------------------------------- 33.8/212.5 MB 42.1 MB/s eta 0:00:05
   ------- -------------------------------- 37.7/212.5 MB 37.5 MB/s eta 0:00:05
   -------- ------------------------------- 46.9/2


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from elasticsearch import Elasticsearch, helpers
from langchain.embeddings import HuggingFaceEmbeddings

es = Elasticsearch("http://localhost:9200")

# 1) Use MiniLM which runs great on 4 GB GPU
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2) Scroll & update loop
scroll = es.search(
    index="vietnamnet",
    body={"query":{"match_all":{}},"_source":["summary","content"]},
    scroll="2m", size=500
)
sid, docs = scroll["_scroll_id"], scroll["hits"]["hits"]

while docs:
    texts = [d["_source"].get("summary","") or d["_source"].get("content","") for d in docs]
    vectors = embedder.embed_documents(texts)  # each is 384-dim

    actions = [
      {"_op_type":"update","_index":doc["_index"],"_id":doc["_id"],"doc":{"embedding":vec}}
      for doc,vec in zip(docs,vectors)
    ]
    helpers.bulk(es, actions)

    scroll = es.scroll(scroll_id=sid, scroll="2m")
    sid, docs = scroll["_scroll_id"], scroll["hits"]["hits"]

print("Done.")


C:\Users\Saphyiera\AppData\Local\Temp\ipykernel_4504\1114164210.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
e:\HUST\20242\IT4930-DS\Project\TrendFinGPT\ds-prj\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]`

Done.


In [3]:
!pip install --upgrade langchain-google-generative-ai

ERROR: Could not find a version that satisfies the requirement langchain-google-generative-ai (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for langchain-google-generative-ai


In [5]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import torch

# 1) Elasticsearch client
es = Elasticsearch("http://localhost:9200")

# 2) Local embedding model (384-dim, fits 4 GB GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
embedder = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# 3) Generation model (runs locally, no creds)
gen = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    device=0 if device=="cuda" else -1,
    do_sample=False
)

def retrieve_docs(query: str, k: int = 5):
    # embed query
    qvec = embedder.encode(query).tolist()
    # k-NN via script_score
    body = {
      "size": k,
      "query": {
        "script_score": {
          "query": { "match_all": {} },
          "script": {
            "source": "cosineSimilarity(params.qvec, 'embedding') + 1.0",
            "params": { "qvec": qvec }
          }
        }
      }
    }
    resp = es.search(index="vietnamnet", body=body)
    return [hit["_source"] for hit in resp["hits"]["hits"]]

def answer(question: str) -> str:
    docs = retrieve_docs(question)
    context = "\n\n".join(f"{str(d)}" for d in docs)
    prompt = (
        f"Use the context below to answer the question concisely.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}\nAnswer:"
    )
    out = gen(prompt)[0]["generated_text"]
    return out.strip()

# — test it —
if __name__ == "__main__":
    print(answer("Nội dung nghị định 52 là gì?"))


Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (37713 > 512). Running this sequence through the model will result in indexing errors


OutOfMemoryError: CUDA out of memory. Tried to allocate 31.79 GiB. GPU 0 has a total capacity of 4.00 GiB of which 2.33 GiB is free. Of the allocated memory 790.00 MiB is allocated by PyTorch, and 110.00 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
import torch

torch.cuda.is_available()

True

In [ ]:
!pip uninstall torch torchvision torchaudio
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

^C


In [10]:
from google import genai
from dotenv import load_dotenv
import os

# load your GEMINI_API_KEY from .env
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

# initialize the Gemini client
client = genai.Client(api_key=api_key)

def answer_with_gemini(question: str, k: int = 5) -> str:
    docs = retrieve_docs(question, k=k)
    context = "\n\n".join(f"{str(d)}" for d in docs)
    print(context)
    print("="*100)
    return client.models.generate_content(
        model="gemini-2.0-flash-lite",
        contents=(
            f"Trả lời câu hỏi sau: “{question}”\n"
            f"Dùng những gì bạn biết kết hợp với thông tin truy vấn được, các câu hỏi bạn không thấy trong phần thông tin liên quan hãy thông báo cho người dùng và cố gắng trả lời bằng dữ liệu bạn có"
            f"Dưới đây là các đoạn thông tin liên quan:\n{context}\n\n"
            "Hãy trả lời ngắn gọn, rõ ràng."
        )
    ).text

# — test it —  
if __name__ == "__main__":
    print(answer_with_gemini("Nội dung nghị định 52 là gì?"))


{'title': 'Tiếp tục giảm thuế giá trị gia tăng về 8% trong 6 tháng năm 2024', 'summary': 'Ngày 28/12, Chính phủ đã ban hành Nghị định quy định chính sách giảm thuế giá trị gia tăng (GTGT) theo Nghị quyết 110/2023/QH15 của Quốc hội.', 'url': 'https://vietnamnet.vn/tiep-tuc-giam-thue-gia-tri-gia-tang-ve-8-trong-6-thang-nam-2024-2232904.html', 'author': '', 'content': 'Theo đó, các nhóm hàng hóa, dịch vụ đang áp dụng mức thuế suất 10% được giảm 2% thuế suất thuếgiá trị gia tăng(VAT) từ ngày 1/1/2024 đến hết ngày 30/6/2024.Thuế giá trị gia tăng tiếp tục giảm còn 8%Mức thuế giảm này không áp dụng với một số hàng hóa, dịch vụ, đã được quy định tại Nghị quyết số 43, gồm: Viễn thông, công nghệ thông tin, hoạt động tài chính, ngân hàng, chứng khoán, bảo hiểm, kinh doanh bất động sản, kim loại, sản phẩm từ kim loại đúc sẵn, sản phẩm khai khoáng (không kể khai thác than), than cốc, dầu mỏ tinh chế, sản phẩm hoá chất, sản phẩm hàng hóa và dịch vụ chịu thuế tiêu thụ đặc biệt.Nghị định này có hiệu l

In [1]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import torch

# 1) Elasticsearch client
es = Elasticsearch("http://localhost:9200")

# 2) Local embedding model (384-dim, fits 4 GB GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
embedder = SentenceTransformer("all-MiniLM-L6-v2", device=device)

def retrieve_docs(query: str, k: int = 5):
    # embed query
    qvec = embedder.encode(query).tolist()
    # k-NN via script_score
    body = {
      "size": k,
      "query": {
        "script_score": {
          "query": { "match_all": {} },
          "script": {
            "source": "cosineSimilarity(params.qvec, 'embedding') + 1.0",
            "params": { "qvec": qvec }
          }
        }
      }
    }
    resp = es.search(index="vietnamnet", body=body)
    return [hit["_source"] for hit in resp["hits"]["hits"]]

from google import genai
from dotenv import load_dotenv
import os

# load your GEMINI_API_KEY from .env
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

# initialize the Gemini client
client = genai.Client(api_key=api_key)

def answer_with_gemini(question: str, k: int = 5) -> str:
    docs = retrieve_docs(question, k=k)
    context = "\n\n".join(f"{str(d)}" for d in docs)
    print(context)
    print("="*100)
    return client.models.generate_content(
        model="gemini-2.0-flash-lite",
        contents=(
            f"Trả lời câu hỏi sau: “{question}”\n"
            f"Dùng những gì bạn biết kết hợp với thông tin truy vấn được, các câu hỏi bạn không thấy trong phần thông tin liên quan hãy thông báo cho người dùng và cố gắng trả lời bằng dữ liệu bạn có"
            f"Dưới đây là các đoạn thông tin liên quan:\n{context}\n\n"
            "Hãy trả lời ngắn gọn, rõ ràng."
        )
    ).text

# — test it —  
if __name__ == "__main__":
    print(answer_with_gemini("Nội dung nghị định 52 là gì?"))

e:\HUST\20242\IT4930-DS\Project\TrendFinGPT\ds-prj\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'title': 'Tiếp tục giảm thuế giá trị gia tăng về 8% trong 6 tháng năm 2024', 'summary': 'Ngày 28/12, Chính phủ đã ban hành Nghị định quy định chính sách giảm thuế giá trị gia tăng (GTGT) theo Nghị quyết 110/2023/QH15 của Quốc hội.', 'url': 'https://vietnamnet.vn/tiep-tuc-giam-thue-gia-tri-gia-tang-ve-8-trong-6-thang-nam-2024-2232904.html', 'author': '', 'content': 'Theo đó, các nhóm hàng hóa, dịch vụ đang áp dụng mức thuế suất 10% được giảm 2% thuế suất thuếgiá trị gia tăng(VAT) từ ngày 1/1/2024 đến hết ngày 30/6/2024.Thuế giá trị gia tăng tiếp tục giảm còn 8%Mức thuế giảm này không áp dụng với một số hàng hóa, dịch vụ, đã được quy định tại Nghị quyết số 43, gồm: Viễn thông, công nghệ thông tin, hoạt động tài chính, ngân hàng, chứng khoán, bảo hiểm, kinh doanh bất động sản, kim loại, sản phẩm từ kim loại đúc sẵn, sản phẩm khai khoáng (không kể khai thác than), than cốc, dầu mỏ tinh chế, sản phẩm hoá chất, sản phẩm hàng hóa và dịch vụ chịu thuế tiêu thụ đặc biệt.Nghị định này có hiệu l

In [2]:
answer_with_gemini("Gia vang ngay hom nay")

{'title': 'Giá vàng trong nước và thế giới đồng loạt giảm', 'summary': 'Giá vàng trong nước hôm nay (10/2) đồng loạt giảm theo xu hướng của giá vàng thế giới.', 'url': 'https://vietnamnet.vn/gia-vang-trong-nuoc-va-the-gioi-dong-loat-giam-2108931.html', 'author': 'Hạnh Nguyên', 'content': 'Giá vàng hôm nay 10/2: Quay đầu lao dốcGiá vàng hôm nay 10/2: Giá vàng thế giới quay đầu giảm, thị trường vàng trong nước ít biến động trong bối cảnh các ngân hàng trung ương thắt chặt chính sách lãi suất.Giá vàng miếng và vàng nhẫn trong nước hôm nay cùng đi xuống theo đà giảm của giá vàng quốc tế.Cụ thể,đầu giờ sáng nay(10/2), giá vàng miếng của Công ty TNHH MTV Vàng bạc đá quý Sài Gòn (SJC) giảm 50 nghìn đồng/lượng ở cả chiều mua vào và bán ra so với kết thúc phiên giao dịch hôm qua.Còn giá vàng miếng của Tập đoàn Vàng bạc đá quý Doji tại Hà Nội giảm 150 nghìn đồng/lượng ở chiều mua và giảm 100 nghìn đồng/lượng ở chiều bán ra; giá vàng miếng của Doji tại TP.HCM giảm 50 nghìn đồng/lượng ở chiều mua 

'Giá vàng hôm nay (7/11/2024):\n\n*   **Giá vàng nhẫn:** Giảm mạnh, mức giảm kỷ lục, từ 4,5-6,3 triệu đồng/lượng (mua) và 4,35-4,6 triệu đồng/lượng (bán).\n*   **Giá vàng miếng SJC:** Giảm mạnh, tổng mức giảm 6 triệu đồng/lượng (mua) và 3,5 triệu đồng/lượng (bán).\n*   **Giá vàng thế giới:** Tăng nhẹ vào cuối ngày, hiện ở mức 2.666,1 USD/ounce.\n\n**Lưu ý:** Giá vàng thay đổi liên tục, thông tin trên dựa trên dữ liệu đến thời điểm 7/11/2024.\n'